In [ ]:
# agents/analyzer.py
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
import numpy as np

class Analyzer:
    """
    LLM-based analyzer: generates a short plan/insight text from state,
    and produces a fixed-size embedding to pass to the RL recommender.
    """
    def __init__(self, llm_name="distilgpt2", embed_model_name="sentence-transformers/all-MiniLM-L6-v2", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        # lightweight LLM for text generation (planner-like)
        self.tokenizer = AutoTokenizer.from_pretrained(llm_name)
        self.llm = AutoModelForCausalLM.from_pretrained(llm_name).to(self.device)
        # sentence transformer like model for embeddings (here using transformers encoder)
        # we fallback to using the LLM's encoder if dedicated embedding model not available
        try:
            self.embed_tokenizer = AutoTokenizer.from_pretrained(embed_model_name)
            self.embed_model = AutoModel.from_pretrained(embed_model_name).to(self.device)
            self.use_embed_model = True
        except Exception:
            self.embed_model = None
            self.use_embed_model = False

    def generate_insight(self, state_repr, max_new_tokens=40, temperature=0.7):
        prompt = f"Context: {state_repr}\nInsight:"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        out = self.llm.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=self.tokenizer.eos_token_id
        )
        text = self.tokenizer.decode(out[0], skip_special_tokens=True)
        insight = text[len(prompt):].strip()
        return insight

    def embed_text(self, text):
        """Return a numpy vector embedding for the given text (fixed-size)."""
        if self.use_embed_model:
            toks = self.embed_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
            with torch.no_grad():
                out = self.embed_model(**toks, return_dict=True)
                emb = out.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
            return emb.astype(np.float32)
        else:
            # fallback: simple token length + bag-of-ids (very cheap)
            toks = self.tokenizer(text, return_tensors="pt").to(self.device)
            ids = toks['input_ids'].squeeze().cpu().numpy()
            vec = np.zeros(128, dtype=np.float32)
            vec[:min(len(ids), 128)] = ids[:128] / 10000.0
            return vec
